# notes 


## TODO
- chart on mobile screen?
 
## scope
Online news with api

## Insights and facts
- Total articles : 2M
- env section: 50k
- tone - in 2021, explainers 548, analysis 939
- contributors https://www.theguardian.com/index/contributors

  
## content type: 
- keyword -- 
- series -- 
- contributor -- 
- tone -- analysis, news, comment, features, explainers, others(blog, search: guardian news content tones) 
- type -- article, liveblog, interative
- blog -- 

## Method
- source: https://open-platform.theguardian.com/

## ideas


# init

In [34]:
# init

import requests as r
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import sys

sys.path.insert(1, '/Users/papanash/workspace/dash/buffaloes/lib')
import html_helper as h 


pio.templates.default = "plotly_white"

key_param = '&api-key=8e63e9af-bf98-4ed9-ba82-0c5db1884969'
years_param = '&from-date=1999-01-01&to-date=2021-10-31'

content_api = 'https://content.guardianapis.com/search?' + key_param
sections_api = 'https://content.guardianapis.com/sections?' + key_param
tags_api = 'https://content.guardianapis.com/tags?section=world&' + key_param

country_api = 'https://content.guardianapis.com/search?' + key_param + years_param + '&tag=world/'

def get_search_total(s_list):
    totals_all = {}
    for i in s_list:
        result = r.get(content_api + '&q='+ i).json().get('response')
        total = result.get('total')
        totals_all[i] = total
        return totals_all

def get_year_total(section = '', tag=''):
    totals_all = {}
    param = ''
    if section:
        param = '&section=' + section
    elif tag:
        param = '&tag=' + tag
    
    for year in list(range(1999, 2022)):
        year_param = '&from-date=' + str(year) + '-01-01' '&to-date=' + str(year) + '-12-31'

        result = r.get(content_api + param + year_param).json()

        total = result.get('response').get('total')
        totals_all[year] = total

    return totals_all

def get_section_total(tag):

    api = 'https://content.guardianapis.com/search?tag=' + tag + key_param

    sections = {}

    total = r.get(api).json().get('response').get('total')
    print('total: ', total)
    for i in range(1, round((total / 200)) + 1):
        results = r.get(api + f'&page-size=200&page={i}').json().get('response').get('results')

        for j in results:
            sid = j.get('sectionId')

            if sid in sections.keys():
                sections[sid] = sections[sid] + 1
            else: 
                sections[sid] = 1

    return sections

def get_totals_by_section(sections):
    totals_all = {}
    for section in sections:
        section_param = '&section=' + section
        result = r.get(content_api + section_param).json()
        total = result.get('response').get('total')
        totals_all[section] = total

    return totals_all

def get_totals_by_country(g_countries):
    res = {}
    for i in g_countries:
        total = r.get(country_api + i).json().get('response').get('total')
        res[i] = total
    return res

def get_all_sections():
    sections_all = []

    result = r.get(sections_api).json()
    result = result.get('response').get('results')

    for i in result: 
        sections_all.append(i['id'])

    return sections_all

def get_all_tags():
   res = []
 
   for i in range(1, 8):
       response = r.get(tags_api + f'&page-size=200&page={i}').json().get('response').get('results')
 
       for i in response:
           res.append(i['id'])
  
   return res

# totals_20y = totals_all.copy()        
# for i in [1999, 2000, 2021]:
    #     totals_20y.pop(i, None) 


In [2]:
# read
df = pd.read_csv('total_articles__year.csv', index_col=0)
df_sections = pd.read_csv('guardian_sections.csv', index_col=0)

In [125]:
# run

# articles in sections and tags by year

sections = ['music', 'politics', 'sport', 'football', 'science', 'environment', 'business']

env_all = get_totals('environment')

df = pd.DataFrame(columns=['year', 'environment'], data={'year': env_all.keys(), 'environment':env_all.values()})
df.set_index('year', inplace=True)


for s in sections:
    totals_all = get_totals_by_year(s)
    df[s]  = totals_all.values()

for tone in ['features', 'news', 'analysis', 'explainers']:
    totals= get_totals_by_year(tag='tone/'+tone)
    df['tone_' + tone] = list(totals.values())

totals_all = get_totals_by_year()
df['all'] = totals_all.values()

# all sections
all_sections = get_totals(get_all_sections())
df1 = pd.DataFrame(columns=['section', 'total'], data={'section': all_sections.keys(), 'total':all_sections.values()})
df_sections = df1[df1.total > 6000].sort_values(by='total', ascending=False).reset_index(drop=True)


# data

In [23]:
df.to_csv('total_articles__year.csv')

In [24]:
df

,environment,music,politics,sport,football,science,tone/features,tone/news,tone/analysis,tone/explainers,all,tone/letters,tone/obituaries,technology/elon-musk,us-news/kamala-harris
year,,,,,,,,,,,,,,,
1999,247,147,3069,2449,4818,391,1453,3642,61,0,52011,1692,997,0,0
2000,494,185,3552,2181,5537,569,1455,5283,197,0,67430,2332,1039,0,0
2001,345,373,5866,4538,5761,343,1461,6306,250,0,84445,2108,1010,0,0
2002,624,1567,5790,7085,7579,332,1689,5404,171,0,89988,2154,989,0,0
2003,802,2781,7249,8087,6855,709,2054,6120,141,0,94623,2058,1055,0,0
2004,877,3403,6563,8185,7748,938,2181,6362,121,0,97873,2090,1052,0,0
2005,984,3482,7180,8270,6885,1211,2170,6276,141,0,99940,2078,1204,0,0
2006,1508,3967,4565,8396,8688,862,2188,7531,347,0,108381,1895,1452,0,0
2007,2067,6823,4829,10105,8169,975,1065,8429,487,0,125865,2100,1376,0,0


In [7]:
df.sum()


environment          48493
music                95244
politics            109755
sport               170786
football            169535
science              23774
tone_features       263616
tone_news           571058
tone_analysis        18596
tone_explainers       1995
all                2302630
dtype: int64

In [44]:
df_sections[:20]

,section,total
0,world,210064
1,sport,171223
2,football,170533
3,business,143217
4,commentisfree,139375
5,uk-news,130591
6,media,127970
7,politics,110355
8,music,95290
9,books,77291


# story: environment

The Guardian has reported more news articles on its website about the environment than other topics like politics, music, etc.

- https://www.theguardian.com/environment/2021/oct/25/the-guardians-climate-pledge
- https://www.theguardian.com/environment/ng-interactive/2019/oct/16/the-guardians-climate-pledge-2019


In [18]:
# env vs politics

def legend_on_top(fig):
    return fig.update_layout(legend=dict(orientation="h", xanchor="right", yanchor="bottom"  , y=0.85, x=1), legend_title_text='')

def add_shape(fig, **kw):
    return fig.add_shape(xref="x", yref="y",line_color="Black", **kw)

# , 'sport', 'football', 'science' # 'music'
color1 = px.colors.qualitative.D3[3]
color2 = px.colors.qualitative.D3[2]
fig = px.bar(df, y=['environment',  'politics'],  barmode='group', color_discrete_sequence=[color2, color1], width=800, hover_name='variable', hover_data={'variable':False})

fig = legend_on_top(fig)


fig = add_shape(fig, type="circle", x0=2020.5, y0=2100, x1=2021.5, y1=2700)
    
title={
    'text': 'Total number of articles published on environment vs politics',
    'y':0.9,
     'x':0.5,
    # 'xanchor': 'center',
    # 'yanchor': 'top'
    }
fig.update_layout(title =  title,xaxis_title='', yaxis_title='News articles')


fig.show()

In [24]:
# export_to_chart_studio
import chart_studio
import chart_studio.plotly as chart

chart_studio.tools.set_credentials_file(username='papanash',
                                        api_key='JmGCF0wSJBNXJCXlMTDg')

chart.plot(fig)


'https://plotly.com/~papanash/14/'

In [29]:
# top sections in 2021

totals_all = {}
for i in df_sections.section.to_list():
    section = '&section=' + i
    
    year_param = '&from-date=' +  '2021-01-01' '&to-date=' +  '2021-10-31'

    result = r.get(api + section + year_param).json()
    total = result.get('response').get('total')
    totals_all[i] = total

df_2021 = pd.DataFrame(columns=['section', 'total'], data={'section': totals_all.keys(), 'total':totals_all.values()})

df2 = df_2021.sort_values(by='total', ascending=False).reset_index(drop=True)
df2 = df2[~df2.section.isin(['commentisfree', 'australia-news', 'us-news'])][:10]
df2

,section,total
0,world,7771
1,football,5007
2,sport,4962
6,business,3315
7,uk-news,2710
8,environment,2405
9,politics,2340
10,lifeandstyle,2282
11,film,2086
12,society,1919


In [30]:
fig = None

s_color = px.colors.qualitative.Plotly[0]
colors = [s_color] * df2.shape[0]
colors[5] =  'green'
colors[6] = px.colors.qualitative.Plotly[1]

y = [ 'UK News' if i.startswith('uk-news')  else i.title() for i in list(df2.section)] 

fig = px.bar(df2, x='total', y=y, orientation='h')
fig.update_traces(marker_color=colors, text=df2.total)

fig.update_layout(yaxis={'categoryorder': 'total ascending'}, height=400, width=800, 
margin=dict(pad=10))

title={
    'text': 'Total number of articles in the top 10 sections in 2021',
    # 'y':0.9,
     'x':0.5,
    # 'xanchor': 'center',
    # 'yanchor': 'top'
    }

fig.update_layout(title =  title,xaxis_title='', yaxis_title='') 



fig.update_yaxes(tickfont=dict(size=16), tickwidth=20, ticklen=30)
fig.update_xaxes(showticklabels=False, showgrid=False)

fig.show()

In [95]:
df_2021.sort_values(by='total', ascending=False).reset_index(drop=True)[:10]

,section,total
0,world,7771
1,football,5007
2,sport,4962
3,commentisfree,4356
4,australia-news,4185
5,us-news,3764
6,business,3315
7,uk-news,2710
8,environment,2405
9,politics,2340


In [6]:
f = px.line(df, y=['environment',  'politics'],   color_discrete_sequence=['green', px.colors.qualitative.Plotly[1]], width=800)



# BBC, Other sources

In [1]:
from newsapi import NewsApiClient
napi = NewsApiClient(api_key='6e793f63321c41e0b4e2f44ae12af198')

# api.get_top_headlines(sources='bbc-news')



In [22]:

napi_env = 'https://newsapi.org/v2/top-headlines?sources=bbc-news&apiKey=6e793f63321c41e0b4e2f44ae12af198&pageSize=100'

response = r.get(napi_env).json().get('response')

In [24]:
r.get(napi_env).json()

{'status': 'ok',
 'totalResults': 10,
 'articles': [{'source': {'id': 'bbc-news', 'name': 'BBC News'},
   'author': 'BBC News',
   'title': 'Travis Scott festival tragedy: Were warning signs missed at Astroworld?',
   'description': "Concerns were raised before Travis Scott's fatal festival show. Could the tragedy have been averted?",
   'url': 'https://www.bbc.co.uk/news/entertainment-arts-59205840',
   'urlToImage': 'https://ichef.bbci.co.uk/news/1024/branded_news/3762/production/_121487141_976_gettyimages-1351663182.jpg',
   'publishedAt': '2021-11-09T09:07:23.504589Z',
   'content': 'By Mark SavageBBC Music Correspondent\r\nImage source, Getty Images\r\nImage caption, Scott was performing in front of his home crowd in Houston, Texas\r\nFor almost 40 minutes after Houston police and fir… [+9723 chars]'},
  {'source': {'id': 'bbc-news', 'name': 'BBC News'},
   'author': 'BBC News',
   'title': "Hundreds of migrants at Belarus' border with Poland",
   'description': 'Poland says it ha

# story: tones/genres
- rise of explainers vs fall of anaylsis since 2015
- letters - what happened in 2009, 2010, 2020

In [38]:
px.bar(df, y=['tone_letters'], width=800)


In [29]:
px.line(df, y=['tone_analysis', 'tone_features', 'tone_explainers'], width=800)


# story: country representation

- iraq + afghan > Africa
- middleeast > africa, americas, south-and-central-asia
- most reported countries (total, per-capita)

In [ ]:
a = 'https://content.guardianapis.com/tags?section=world&from-date=1999-01-01&to-date=2021-10-31&page-size=200&api-key=8e63e9af-bf98-4ed9-ba82-0c5db1884969&type=keyword&page='

res = {}

for i in range(1, 7):
    response = r.get(a + str(i)).json().get('response')
    
    print(response.get('total'))

    response =  response.get('results')

    for i in response:
        res[i['id']] = i['type']

res
v = list(res.keys())
v = [i[6:] for i in v]

exist2 = []
not_exist2 = []
for p, i in enumerate(world):
    if not ' (' in i: # ' and' in i and  and len(i.split(' ')) > 1:
        if i.replace(' ', '').lower() in v:
            exist2.append(i.replace(' ', '').lower())
        
        elif i.replace(' ', '-').lower() in v:
            exist2.append(i.replace(' ', '-').lower())
        else:
            not_exist2.append(i)

exist2.extend(['state-of-georgia', 'cape-verde', 'palestinian-territories', 'ivory-coast', 'vaticancitystate', 'stlucia', 'congo', 'myanmar', 'czech-republic', 'eswatini', 'congo-brazzaville', 'us','uk', 'au'])
g_countries = exist2

country_total = get_totals_by_country(g_countries)
df_country =  pd.DataFrame(columns=['country', 'total'], data={'country': country_total.keys(), 'total':country_total.values()})

df_country.to_csv('g_countries.csv')

df_country = df_country.append({'place':'europe-news', 'total':86892, 'located_in':'world', 'type':'region'}, ignore_index=True)


g_regions = 'middleeast', 'africa', 'americas', 'south-and-central-asia', 'asia-pacific'

In [324]:
df_country.sort_values(by='total', ascending=False)[:20]


,country,total
71,iraq,22457
33,china,18520
53,france,16839
132,russia,16300
73,israel,16192
0,afghanistan,12364
56,germany,10648
157,syria,10473
182,palestinian-territories,9486
74,italy,9374


In [326]:
africa = [
'Nigeria'
,'Ethiopia'
,'Egypt'
,'DR Congo'
,'Tanzania'
,'South Africa'
,'Kenya'
,'Uganda'
,'Algeria'
,'Sudan'
,'Morocco'
,'Angola'
,'Mozambique'
,'Ghana'
,'Madagascar'
,'Cameroon'
,'Côte Ivoire'
,'Niger'
,'Burkina Faso'
,'Mali'
,'Malawi'
,'Zambia'
,'Senegal'
,'Chad'
,'Somalia'
,'Zimbabwe'
,'Guinea'
,'Rwanda'
,'Benin'
,'Burundi'
,'Tunisia'
,'South Sudan'
,'Togo'
,'Sierra Leone'
,'Libya'
,'Congo'
,'Liberia'
,'Central African Republic'
,'Mauritania'
,'Eritrea'
,'Namibia'
,'Gambia'
,'Botswana'
,'Gabon'
,'Lesotho'
,'Guinea-Bissau'
,'Equatorial Guinea'
,'Mauritius'
,'Eswatini'
,'Djibouti'
,'Comoros'
,'Cabo Verde'
,'Sao Tome & Principe'
,'Seychelles'
]


In [340]:
def match_countries():
    exist2 = []
    not_exist2 = []
    for p, i in enumerate(africa):
        if not ' (' in i: # ' and' in i and  and len(i.split(' ')) > 1:
            if i.replace(' ', '').lower() in v:
                exist2.append(i.replace(' ', '').lower())
            
            elif i.replace(' ', '-').lower() in v:
                exist2.append(i.replace(' ', '-').lower())
            else:
                not_exist2.append(i)
    # 'cape-verde',  'ivory-coast', 'congo','sao-tome-and-principe'
    return not_exist2, exist2




In [381]:
df_country[df_country.country.isin(['iraq', 'afghanistan'])].total.sum()

34821

In [379]:
r.get(country_api + 'africa').json().get('response').get('total')

30296

In [418]:
df_country[df_country.type == 'region']

,place,total,located_in,type
196,middleeast,63514,world,region
197,africa,30296,world,region
198,americas,18885,world,region
199,south-and-central-asia,8779,world,region
200,asia-pacific,34472,world,region
201,europe-news,86892,world,region


In [426]:
a = 'https://content.guardianapis.com/search?tag=world%2Fsrilanka&from-date=2020-12-01&to-date=2020-12-31&show-tags=all&page-size=200&api-key=8e63e9af-bf98-4ed9-ba82-0c5db1884969'
response = r.get(a).json().get('response')

In [427]:
response = response.get('results')

for i in response:
    tags = i.get('tags')
    for t in tags:
        print(t.get('id'), t.get('type'))
    print('----')

world/srilanka keyword
world/islam keyword
world/coronavirus-outbreak keyword
world/south-and-central-asia keyword
world/world keyword
world/religion keyword
world/world-health-organization keyword
science/infectiousdiseases keyword
type/article type
tone/news tone
profile/harrietsherwood contributor
----
film/film keyword
film/drama keyword
world/srilanka keyword
world/lgbt-rights keyword
world/world keyword
world/south-and-central-asia keyword
culture/culture keyword
film/ava-duvernay keyword
type/article type
tone/reviews tone
profile/cathclarke contributor
tracking/commissioningdesk/uk-culture tracking
----
australia-news/series/lives-in-limbo series
australia-news/australian-immigration-and-asylum keyword
australia-news/manus-island keyword
australia-news/reza-barati keyword
australia-news/australia-news keyword
australia-news/melbourne keyword
world/srilanka keyword
type/article type
tone/features tone
profile/green-michael contributor
publication/theguardian publication
theguard

# country explorer
- top 5 doesn't seem interesting without comparison. can take >5 seconds from api. 

In [150]:
sorted(pakistan.items(), key=lambda x: x[1], reverse=True)[:10]


[('world', 4400),
 ('commentisfree', 932),
 ('uk-news', 201),
 ('us-news', 169),
 ('global-development', 163),
 ('media', 160),
 ('politics', 98),
 ('environment', 84),
 ('books', 62),
 ('sport', 57)]

In [151]:
srilanka = get_section_total('world/srilanka')
sorted(srilanka.items(), key=lambda x: x[1], reverse=True)[:10]


total:  1252


[('world', 667),
 ('commentisfree', 143),
 ('australia-news', 87),
 ('global-development', 47),
 ('uk-news', 34),
 ('media', 31),
 ('politics', 26),
 ('environment', 19),
 ('sport', 18),
 ('books', 17)]

In [152]:
sorted(india.items(), key=lambda x: x[1], reverse=True)[:10]


[('world', 4169),
 ('commentisfree', 1005),
 ('global-development', 553),
 ('business', 540),
 ('environment', 489),
 ('film', 337),
 ('books', 193),
 ('politics', 157),
 ('sport', 154),
 ('cities', 149)]

In [153]:
sweden = get_section_total('world/sweden')
sorted(sweden.items(), key=lambda x: x[1], reverse=True)[:10]

total:  1718


[('world', 518),
 ('media', 245),
 ('commentisfree', 207),
 ('business', 97),
 ('books', 81),
 ('music', 63),
 ('environment', 58),
 ('film', 51),
 ('technology', 47),
 ('society', 35)]

In [154]:
italy = get_section_total('world/italy')
sorted(italy.items(), key=lambda x: x[1], reverse=True)[:10]

total:  9406


[('world', 5717),
 ('commentisfree', 774),
 ('business', 670),
 ('film', 251),
 ('artanddesign', 219),
 ('uk-news', 203),
 ('media', 167),
 ('politics', 150),
 ('fashion', 148),
 ('us-news', 141)]

# celebrities
obituaries, rich people attention, etc

Innovators
Jensen Huang
Elon Musk
Adrienne Banfield Norris, Willow Smith, & Jada Pinkett Smith
Katalin Kariko
Mary Barra
John Nkengasong
MiMi Aung
Vitalik Buterin
Viya
Barney Graham
Friederike Otto & Geert Jan van Oldenborgh
Kengo Kuma
Sara Menker
Lidia Morawska

In [32]:
times_leaders = ['Ngozi Okonjo-Iweala',
'Joe Biden',
'Xi Jinping',
'Liz Cheney',
'Kamala Harris',
'Mario Draghi',
'Tucker Carlson',
'Naftali Bennett',
'Stacey Abrams',
'Nayib Bukele',
'Donald Trump',
'Narendra Modi',
'Mahbouba Seraj',
'Joe Manchin',
'Ebrahim Raisi',
'Rochelle Walensky',
'Mamata Banerjee',
'Ron Klain',
'Elisa Loncon Antileo',
'Abdul Ghani Baradar']



In [35]:
totals_all

{'Ngozi Okonjo-Iweala': 857,
 'Joe Biden': 26272,
 'Xi Jinping': 16075,
 'Liz Cheney': 19458,
 'Kamala Harris': 12547,
 'Mario Draghi': 16218,
 'Tucker Carlson': 5553,
 'Naftali Bennett': 16047,
 'Stacey Abrams': 5713,
 'Nayib Bukele': 46,
 'Donald Trump': 51982,
 'Narendra Modi': 2735,
 'Mahbouba Seraj': 23,
 'Joe Manchin': 16382,
 'Ebrahim Raisi': 665,
 'Rochelle Walensky': 1237,
 'Mamata Banerjee': 363,
 'Ron Klain': 18442,
 'Elisa Loncon Antileo': 590,
 'Abdul Ghani Baradar': 8019}

In [47]:
pd.DataFrame.from_dict({'times_leaders': totals_all.keys(), 'total':totals_all.values()}).sort_values(by='total', ascending=False).reset_index(drop=True)

,times_leaders,total
0,Donald Trump,51982
1,Joe Biden,26272
2,Liz Cheney,19458
3,Ron Klain,18442
4,Joe Manchin,16382
5,Mario Draghi,16218
6,Xi Jinping,16075
7,Naftali Bennett,16047
8,Kamala Harris,12547
9,Abdul Ghani Baradar,8019
